In [ ]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="svm", 
           entity="benchmark-nlp",
           name='topic datasets svm') #CHANGE

In [2]:
import os 
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42 
OPT_ITER=10

## Load data

In [5]:
dl = DataLoader(['topic'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization
train_twentynews, val_twentynews, _ = data_splitter(data['twentynews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_agnews, val_agnews, _ = data_splitter(data['agnews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_yahoo, val_yahoo, _ = data_splitter(data['yahoo'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

1400000 rows preprocessed in 1108.4388880729675 seconds
60000 rows preprocessed in 37.829808473587036 seconds


In [ ]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

In [8]:
embedded_train_twentynews = fasttext.generate_sentence_embeddings(train_twentynews['text'])
embedded_val_twentynews = fasttext.generate_sentence_embeddings(val_twentynews['text'])
embedded_train_twentynews['label'] = train_twentynews['label'].to_list()
embedded_val_twentynews['label'] = val_twentynews['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 9051/9051 [01:25<00:00, 105.50it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 2263/2263 [00:20<00:00, 111.76it/s]


In [9]:
embedded_train_agnews = fasttext.generate_sentence_embeddings(train_agnews['text'])
embedded_val_agnews = fasttext.generate_sentence_embeddings(val_agnews['text'])
embedded_train_agnews['label'] = train_agnews['label'].to_list()
embedded_val_agnews['label'] = val_agnews['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 96000/96000 [05:22<00:00, 297.56it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 24000/24000 [01:19<00:00, 302.99it/s]


In [ ]:
embedded_train_yahoo = fasttext.generate_sentence_embeddings(train_yahoo['text'])
embedded_val_yahoo = fasttext.generate_sentence_embeddings(val_yahoo['text'])
embedded_train_yahoo['label'] = train_yahoo['label'].to_list()
embedded_val_yahoo['label'] = val_yahoo['label'].to_list()

## Hyperopt

In [6]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/svm_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [7]:
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'C': {'min': 0, 'max': 10, 'distribution': 'uniform'},
  'kernel': {'values': ['linear', 'rbf']},
  'probability': {'value': True},
  'random_state': {'value': 42}}}

#### twenty news   Tf-Idf

In [13]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_twentynews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_twentynews, #Change here
          val=val_twentynews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = SVC(C=config.C, 
                  kernel=config.kernel,
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER) #Count : number of iterations
tracker.stop()

[codecarbon WARNING @ 17:45:22] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 11n0o4a9
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/11n0o4a9


[codecarbon WARNING @ 17:45:25] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: vnu3eed7 with config:
wandb: 	C: 9.055592611126652
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.71763
f1 macro,0.7277


wandb: Agent Starting Run: qn05gumg with config:
wandb: 	C: 9.283720563215184
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: lkkvohg8 with config:
wandb: 	C: 7.589747097587118
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.72249
f1 macro,0.73061


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ahtbycf6 with config:
wandb: 	C: 0.6755032005930905
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.74459
f1 macro,0.73787


0.0024518219856565584

In [14]:
#Don't forget to name the sweep instance   
name = 'svm_ft_twentynews' #change here
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_twentynews, #Change here
          val=embedded_val_twentynews): #change here
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        clf = SVC(C=config.C, 
                  kernel=config.kernel,
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 18:01:48] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: in0jrs99
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/in0jrs99


[codecarbon WARNING @ 18:01:51] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: nnvk2rt7 with config:
wandb: 	C: 7.536436517030978
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.66019
f1 macro,0.64632


wandb: Agent Starting Run: e99d3hjf with config:
wandb: 	C: 1.179566674641617
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: 9xz86b69 with config:
wandb: 	C: 4.681814701268769
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.67565
f1 macro,0.66563


wandb: Agent Starting Run: y0pgnaoy with config:
wandb: 	C: 7.125864823569228
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.65886
f1 macro,0.64377


0.001348667556281457

#### agnews tfidf

In [8]:
#Don't forget to name the sweep instance  
name = 'linearsvc_tfidf_agnews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_agnews, #Change here
          val=val_agnews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = LinearSVC(C=config.C, 
#                  kernel=config.kernel,
                   loss='hinge',
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

Create sweep with ID: muh2dn51
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/muh2dn51


[codecarbon WARNING @ 10:41:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:41:53] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 8c16uixc with config:
wandb: 	C: 9.174144641053768
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.907
f1 macro,0.90668


wandb: Agent Starting Run: wsxukona with config:
wandb: 	C: 1.8434737660354104
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.92087
f1 macro,0.92051


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6dsndypq with config:
wandb: 	C: 2.182682146328209
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.92046
f1 macro,0.92009


wandb: Agent Starting Run: o0fjj14c with config:
wandb: 	C: 3.273306999215996
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.91904
f1 macro,0.91867


0.0015805391262487434

In [10]:
#Don't forget to name the sweep instance   
name = 'svm_ft_agnews' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_agnews, #CHANGE HERE
          val=embedded_val_agnews): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = LinearSVC(C=config.C, 
#                   kernel=config.kernel,
                    loss='hinge',
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        aucpc = '-'
        auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

Create sweep with ID: k3rhn1td
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/k3rhn1td


[codecarbon WARNING @ 10:59:21] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:59:24] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 1gyeqld5 with config:
wandb: 	C: 0.06512241722822276
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.87858
f1 macro,0.87776


wandb: Agent Starting Run: 3ef3ofcn with config:
wandb: 	C: 3.726405068697055
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88879
f1 macro,0.88827


...

wandb: Agent Starting Run: 3907pm8d with config:
wandb: 	C: 1.1547981661943352
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88796
f1 macro,0.8874


wandb: Agent Starting Run: cgdvppv3 with config:
wandb: 	C: 9.337129148384438
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.88938
f1 macro,0.88887


0.0013698724748339576

In [12]:
del embedded_train_agnews
del embedded_val_agnews

#### yahoo

In [8]:
#Don't forget to name the sweep instance  
name = 'svm_tfidf_yahoo' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_tfidf(config = None,
          train=train_yahoo, #Change here
          val=val_yahoo): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = LinearSVC(C=config.C, 
#                   kernel=config.kernel,
                    loss='hinge',
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
#         pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#         else:
        aucpc = '-'
        auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 10:00:32] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: ti974o5i
Sweep URL: https://wandb.ai/benchmark-nlp/svm/sweeps/ti974o5i


[codecarbon WARNING @ 10:00:34] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: w93y1ag1 with config:
wandb: 	C: 4.359134783303472
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.4361
f1 macro,0.48845


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: jdih6vw2 with config:
wandb: 	C: 0.13586381752014876
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


...

wandb: Agent Starting Run: p9yuggkj with config:
wandb: 	C: 6.661346461500591
wandb: 	kernel: rbf
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43252
f1 macro,0.4844


wandb: Agent Starting Run: 0ublolaq with config:
wandb: 	C: 6.420682518349638
wandb: 	kernel: linear
wandb: 	probability: True
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43284
f1 macro,0.48479


0.023820712064613857

In [16]:
#Don't forget to name the sweep instance   
name = 'svm_ft_yahoo' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="svm")

def train_fasttext(config = None,
          train=embedded_train_yahoo, #CHANGE HERE
          val=embedded_val_yahoo): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = LinearSVC(C=config.C, 
#                   kernel=config.kernel,
                    loss='hinge',
#                   probability=config.probability,
                  random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
#         pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
#         if train['label'].nunique() <=2:
#             aucpc =  average_precision_score(val['label'],pred_prob_val)
#             auc = roc_auc_score(val['label'],pred_prob_val)
#             #Log predictions on WandB
#         else:
        aucpc = '-'
        auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:14:08] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 8q83yjoe
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/8q83yjoe


[codecarbon WARNING @ 15:14:11] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: lzriiexe with config:
wandb: 	C: 3.426147217376755
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.42979
f1 macro,0.47649


wandb: Agent Starting Run: r2tymvzh with config:
wandb: 	C: 5.357984774600081
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


...

wandb: Agent Starting Run: pjlw5f0k with config:
wandb: 	C: 2.6710627849389743
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43143
f1 macro,0.47794


wandb: Agent Starting Run: 7lwpcbgk with config:
wandb: 	C: 8.537706451301336
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43017
f1 macro,0.47666


0.006280692263080593